## 掛接google drive開始訓練

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##出現提示欄進行授權
import os

os.chdir('/content/drive/My Drive/darknet')
os.listdir() #確認目錄內容

##確認使用顯卡

In [3]:
! nvidia-smi

Fri May 28 06:31:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##產出訓練與測試資料的位置txt文件

In [7]:
#Segmentation fault(core dumped) debug
# 輸出 train.txt
!cd data/train;ls -d "$PWD"/*.jpg > train.txt
# 輸出 dev.txt
!cd data/dev;ls -d "$PWD"/*.jpg > dev.txt


##修改Makefile文件

In [ ]:
!sed -i "s/GPU=0/GPU=1/g" darknet/Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" darknet/Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" darknet/Makefile

!head darknet/Makefile

GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0



##初始化darknet

In [10]:
!make

/bin/bash: line 0: cd: darknet: No such file or directory
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include obj/image_opencv.o obj/http_stream.o obj/gemm.o obj/utils.o obj/dark_cuda.o obj/convolutional_layer.o obj/list.o obj/image.o obj/activations.o obj/im2col.o obj/col2im.o obj/blas.o obj/crop_layer.o obj/dropout_layer.o obj/maxpool_layer.o obj/softmax_layer.o obj/data.o obj/matrix.o obj/network.o obj/connected_layer.o obj/cost_layer.o obj/parser.o obj/option_list.o obj/darknet.o obj/detection_layer.o obj/captcha.o obj/route_layer.o obj/writing.o obj/box.o obj/nightmare.o obj/normalization_layer.o obj/avgpool_layer.o obj/coco.o obj/dice.o obj/yolo.o obj/detector.o obj/layer.o obj/compare.o obj/classifier.o obj/local_l

##開始訓練  
大資料集avg_loss約為3即可停止訓練  
小資料集avg_loss約為0.05即可停止訓練  
訓練完成的weight會存在backup  

In [ ]:
!ls
!chmod +x darknet;./darknet detector train data/train.data cfg/train.cfg yolov4.conv.137 -dont_show

##執行預測  
注意weight檔案名稱是否正確

In [ ]:
!ls
!chmod +x darknet;./darknet detector test data/train.data cfg/train.cfg backup/train_1545_loss025.weights data/test.jpg